# Name of UQ Method

# Theoretic Foundation / Papers where it was introduced

# Imports

In [ ]:
import os
import sys

# required to make the path visible to import the tools
# this will change in public notebooks to be "pip install uq-regression-box"
nb_dir = os.path.split(os.path.split(os.getcwd())[0])[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
nb_dir

import tempfile
from uq_method_box.datamodules import ToyHeteroscedasticDatamodule
from uq_method_box.models import MLP
from uq_method_box.eval_utils import plot_training_metrics
from uq_method_box.viz_utils import plot_predictions, plot_calibration_uq_toolbox
import torch.nn as nn
from lightning import Trainer
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import CSVLogger

In [ ]:
seed_everything(0)  # seed everything for reproducibility

We define a temporary directory to look at some training metrics and results.

In [ ]:
my_temp_dir = tempfile.mkdtemp()

# Datamodule

To demonstrate the method, we will make use of a Toy Regression Example that is defined as a [Lightning Datamodule](https://lightning.ai/docs/pytorch/stable/data/datamodule.html). While this might seem like overkill for a small toy problem, we think it is more helpful how the individual pieces of the library fit together so you can train models on more complex tasks.

In [ ]:
dm = ToyHeteroscedasticDatamodule()

X_train, y_train, train_loader, X_test, y_test, test_loader = (
    dm.X_train,
    dm.y_train,
    dm.train_dataloader(),
    dm.X_test,
    dm.y_test,
    dm.test_dataloader(),
)

# Model

For our Toy Regression problem, we will use a simple Multi-layer Perceptron (MLP) that you can configure to your needs. For the documentation of the MLP see [here](https://torchgeo.readthedocs.io/en/stable/api/models.html#MLP).

In [ ]:
network = MLP(n_inputs=1, n_hidden=[50], n_outputs=1, activation_fn=nn.Tanh())
network

With an underlying neural network, we can now use our desired UQ-Method as a sort of wrapper. All UQ-Methods are implemented as [LightningModule](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html) that allow us to concisely organize the code and remove as much boilerplate code as possible.

In [ ]:
# uq_method = ...

# Trainer

Now that we have a LightningDataModule and a UQ-Method as a LightningModule, we can conduct training with a [Lightning Trainer](https://lightning.ai/docs/pytorch/stable/common/trainer.html). It has tons of options to make your life easier, so we encourage you to check the documentation.

In [ ]:
logger = CSVLogger(my_temp_dir)
trainer = Trainer(
    max_epochs=200,  # number of epochs we want to train
    logger=my_temp_dir,  # log training metrics for later evaluation
    log_every_n_steps=1,
    checkpoint_callback=False,
)

Training our model is now easy:

In [ ]:
trainer.fit()  # (uq_method, dm)

# Training Metrics

To get some insights into how the training went, we can use the utility function to plot the training loss and RMSE metric.

In [ ]:
fig = plot_training_metrics(my_temp_dir)

# Prediction

For prediction we can either rely on the `trainer.test()` method or manually conduct a `predict_step()`. Using the trainer will save the predictions and some metrics to a CSV file, while the manual `predict_step()` with a single input tensor will generate a dictionary that holds the mean prediction as well as some other quantities of interest, for example the predicted standard deviation or quantile.

In [ ]:
# save predictions
trainer.test()  # (uq_method, dm.test_dataloader())

In [ ]:
# preds = uq_method.predict_step(X_test)

# Evaluate Predictions

In [ ]:
fig = plot_predictions()

For some additional metrics relevant to UQ, we can use the great [uncertainty-toolbox](https://uncertainty-toolbox.github.io/) that gives us some insight into the calibration of our prediction. For a discussion of why this is important, see ...

In [ ]:
fig = plot_calibration_uq_toolbox()

# Additional Resources

Links to othere related literature that might be interesting.